# Faker for testing databases
Learn to use the `faker` module to allow testing database designs.

In [1]:
from faker import Faker

ModuleNotFoundError: No module named 'faker'

In [ ]:
faker = Faker()

In [ ]:
faker.color_name()

In [ ]:
print(faker.credit_card_full())

In [ ]:
faker.date_of_birth()

In [ ]:
help(faker.ean13)

In [ ]:
faker.phone_number()

In [ ]:
print(faker.paragraph())

In [ ]:
faker.license_plate()

In [ ]:
faker.street_address()

In [ ]:
faker.coordinate()

In [ ]:
dir(faker)

In [ ]:
faker.words(4)

## Learn about Cursors
Cursors are the usual way of issuing database queries and processing their results.

In [ ]:
import json
with open('cred.json') as f:
    creds = json.load(f)

In [ ]:
list(creds)

In [ ]:
import pymysql

# establish a database connection
conn = pymysql.connect(host=creds['host'], user=creds['user'], passwd=creds['password'])

In [ ]:
cursor = conn.cursor(cursor=pymysql.cursors.DictCursor)

In [ ]:
cursor.execute("""
CREATE SCHEMA dimitri_test
""")

In [ ]:
cursor.execute("""
CREATE TABLE dimitri_test.fake_person(
person_id int not NULL,
first_name varchar(30) NOT NULL,
last_name varchar(30) NOT NULL,
date_of_birth date NOT NULL,
primary key(person_id)
)
"""
)

In [ ]:
# insert 
cursor.execute("""INSERT INTO dimitri_test.fake_person (person_id, first_name, last_name, date_of_birth) VALUES
(%s, %s, %s, %s)
""", (3, faker.first_name(), faker.last_name(), faker.date_of_birth()))

In [ ]:
import tqdm

In [ ]:
for i in tqdm.tqdm(range(0, 200)):
    cursor.execute("""
    INSERT INTO 
    dimitri_test.fake_person (person_id, first_name, last_name, date_of_birth) 
    VALUES (%s, %s, %s, %s)
    """, (i, faker.first_name(), faker.last_name(), faker.date_of_birth()))

In [ ]:
cursor.execute("""
SELECT * FROM dimitri_test.fake_person
""")

In [ ]:
cursor.fetchall()

In [ ]:
cursor.execute("""
DROP TABLE dimitri_test.fake_person
""")

In [ ]:
import datetime

In [ ]:
cursor.execute("""use dimitri_test""")

In [ ]:
cursor.execute("""SELECT * FROM fake_person""")

In [ ]:
cursor.fetchone()

In [ ]:
cursor.fetchall()

In [ ]:
cursor.execute("""SELECT * FROM dimitri_test.fake_person""")
for rec in cursor:
    print(rec)

In [ ]:
import datetime

In [ ]:
faker.date_between(datetime.date(2018, 2, 3), 'today')

In [ ]:
cursor.execute("""
drop table fake_death
""")

In [ ]:
cursor.execute("""
CREATE TABLE dimitri_test.fake_death(
    person_id int not null,
    date_of_death date NOT NULL,
    primary key(person_id), 
    foreign key (person_id) REFERENCES dimitri_test.fake_person (person_id))
""")

In [ ]:
cursor.execute("""
CREATE TABLE hotel_reserviation(
    
    hotel varchar(20) not null
    room  int not null,
    reservation_date date,
    person_id int not null,
    
    unique index (person_id, reservation_date),    
    primary key (hotel, room, reservation_date),
    foreign key (person_id) references fake_person(person_id)
""")

In [ ]:
cursor.execute("""
CREATE TABLE bank_account (
    
    bank_id  int not null, 
    account int not null,
    
    primary key(bank_id, account)

""")

cursor.execute("""
CREATE TABLE bank_account_owner (
    
    bank_id  int not null, 
    account int not null,
    person_id int not null,
    
    primary key(bank_id, account, person_id),
    foreign key (person_id) references fake_person(person_id),
    foreign key (bank_id, account) references fake_person(bank_id, account),

""")

In [ ]:
cursor.execute("""
SELECT * FROM fake_person
""")
cursor.fetchone()

In [ ]:
cursor.execute("""INSERT into fake_death (date_of_death) values ('2020-10-09')""")

In [ ]:
cursor.execute("""INSERT into fake_death (person_id, date_of_death) values (1000, '2020-09-09')""")

In [ ]:
persons = cursor.execute("""SELECT person_id, date_of_birth FROM dimitri_test.fake_person""")
for rec in cursor.fetchall():
    cursor.execute("""
    INSERT INTO dimitri_test.fake_death (person_id, date_of_death) VALUES (%s, %s)
    """, (rec['person_id'], faker.date_between(rec['date_of_birth'], rec['date_of_birth'] + datetime.timedelta(days=40000))))
    

In [ ]:
cursor.execute("""
SELECT first_name, floor(DATEDIFF(date_of_death, date_of_birth)/365.25) as died_at
FROM dimitri_test.fake_person NATURAL JOIN dimitri_test.fake_death""")

for rec in cursor:
    print(rec)

In [ ]:
cursor.execute("""
DROP TABLE dimitri_test.fake_death
""")

## Terminology

Translation from relational terminology into database programming

* Tuple -> Row
* Attribute -> Field/column
* Attribute value -> cell
* Relation -> Table
* Domain -> data type